In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import datetime

In [2]:
def convert_mean_seconds(mean_seconds):
        
    H = int(mean_seconds) // 3600
    M = (int(mean_seconds) - H * 3600) // 60
    S = int(mean_seconds) - H * 3600 - M * 60
    
    if len(str(H)) > 1:
        H = str(H)
    else:
        H = '0' + str(H)
    
    if len(str(M)) > 1:
        M = str(M)
    else:
        M = '0' + str(M)
    
    if len(str(S)) > 1:
        S = str(S)
    else:
        S = '0' + str(S)
    
    
    return f'{H}:{M}:{S}'

In [4]:
# load excel

main_excel_path = 'SFT 2022-10.xlsx'
reference_excel_path = '202209-12 製令排程.xlsx'
main_df = pd.read_excel(main_excel_path, header = 2)#表頭為2 從第三行開始讀數據
ref_df = pd.read_excel(reference_excel_path)
main_df.columns = main_df.loc[1, :]#從第貳行當作行名

In [5]:
# delete the row of '小計' 
delete_index_list = main_df[main_df['製程名稱'] == '小        計  ：'].index
#delete_index_list變數，尋找名稱為製程名稱的欄位當中值為小計的行
main_df = main_df.drop(delete_index_list)
#drop掉

In [6]:
#  add new column '生產線別'
main_df['生產線別'] = ''

production_line_index_list = []
for index in tqdm(main_df.index):
    
    temp_str = main_df.loc[index, '生產日期']
    
    if temp_str == '生產線別：':
        temp_production_line = main_df.loc[index, '單號']
        production_line_index_list.append(index)
        
        continue
    
    elif temp_str == '生產日期':
        production_line_index_list.append(index)
        
        continue
    
    main_df.loc[index, '生產線別'] = temp_production_line
        
main_df = main_df.drop(production_line_index_list)

100%|██████████| 34529/34529 [00:05<00:00, 6885.16it/s]


In [7]:
#  add new column '預計產量'
main_df['預計產量'] = 0

for index in tqdm(main_df.index):
    
    temp_order = main_df.loc[index, '製令編號']
    
    estimated_output = ref_df[ref_df['製令單號\nMO No'] == temp_order]['預計產量\nQty']
    
    if estimated_output.values.size == 0:
        continue
    elif np.isnan(estimated_output.values[0]):
        continue
    else:
        main_df.loc[index, '預計產量'] = str(int(estimated_output.values[0]))

100%|██████████| 34519/34519 [01:06<00:00, 518.28it/s]


In [8]:
#  add new column '人工用時1'
#測試用
main_df['使用人時1'] = 'time'

order_list = np.unique(main_df.loc[:, '單號'])

delete_index_list = []

for i_order in tqdm(order_list):
    
    temp_df = main_df[main_df['單號'] == i_order]
    temp_df = temp_df.sort_values(by=['單號'])
    
    timelist = [temp_df['使用人時'].values[0]]
    total_time = datetime.timedelta()
    serial_number = int(temp_df['序號'].values[0])
    for i in timelist:
        (h, m, s) = i.split(':')
        d = datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(s))
        total_time += d
    
   #mean_time = total_time / len(timelist) * serial_number 
   #mean_seconds = convert_mean_seconds(mean_time.total_seconds())
    if temp_df['預計產量'].values[0] == 0:
        mean_seconds = '無預計產量，無法計算'
    else:
        estimated_time_per_product = total_time / len(timelist) * serial_number  / int(temp_df['預計產量'].values[0])
        mean_seconds = convert_mean_seconds(estimated_time_per_product.total_seconds())
    
    main_df.loc[temp_df.index, '使用人時1'] = mean_seconds
    
    
    if len(temp_df) > 1:
        delete_index_list.append(temp_df.index[:-1].values[0])
    
main_df = main_df.drop(delete_index_list)

100%|██████████| 31270/31270 [02:25<00:00, 215.18it/s]


In [9]:
# rearrange order of columns

columns_list = list(main_df.columns)

columns_list.remove('生產線別')
columns_list.remove('預計產量')
columns_list.remove('使用人時1')

target_poc = main_df.columns.get_loc('使用人時')

columns_list.insert(target_poc + 1, '使用人時1')
columns_list.insert(target_poc + 1, '預計產量')
columns_list.insert(0, '生產線別')

main_df = main_df[columns_list]


for i_line in np.unique(main_df['生產線別']):
    temp_df = main_df[main_df['生產線別'] == i_line]
    main_df.loc[temp_df.index[1:], '生產線別'] = ''

In [10]:
main_df.to_excel('10_merged.xlsx', index = False)

In [ ]:
#  add new column '人工用時1'
#  backup
main_df['使用人時1'] = 'time'

order_list = np.unique(main_df.loc[:, '單號'])

delete_index_list = []

i_order = 'D402-20221200287'
for i_order in tqdm(order_list):
    
    temp_df = main_df[main_df['單號'] == i_order]
    temp_df = temp_df.sort_values(by=['單號'])

    total_time = datetime.timedelta()
    (h, m, s) = temp_df['使用人時'].values[0].split(':')
    d = datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(s))
    total_time += d
    
    if temp_df['預計產量'].values[0] == 0:
        mean_seconds = '無預計產量，無法計算'
    else:
        estimated_time_per_product = (len(temp_df) * total_time) / int(temp_df['預計產量'].values[0])
        mean_seconds = convert_mean_seconds(estimated_time_per_product.total_seconds())
    
    main_df.loc[temp_df.index, '使用人時1'] = mean_seconds
    
    ### 
    if len(temp_df) > 1:
        delete_index_list.append(temp_df.index[:-1].values[0])
    
main_df = main_df.drop(delete_index_list)
